In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from bokeh.server.server import Server as server
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON
import bokeh
import pandas as pd
import os
import sys
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.plotting import ColumnDataSource, Figure
from bokeh.models import Title
import bokeh.layouts as layout
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler


# In[2]:


# def path( file):
#     """
#     Set path settings.

#     :param file: file name in format '/dir/name.type'(/dir optional).

#     """
#     if getattr(sys, 'frozen', False):
#         s = os.path.join(os.path.dirname(sys.executable) + file)
#         return s
#     else:
#         s = file
#         return s


# In[3]:


def make_document(doc):
    
    
    
    
    def path( file):
        """
        Set path settings.

        :param file: file name in format '/dir/name.type'(/dir optional).

        """
        if getattr(sys, 'frozen', False):
            s = os.path.join(os.path.dirname(sys.executable) + file)
            return s
        else:
            s = file
            return s
    


    #матрица
    matrix = pd.read_csv(path('onoffmatrix_avg.csv'), sep = ';', encoding='cp1251')
    matrix = matrix[matrix['hour_on']==7]
    matrix = matrix[['stop_id_from','stop_id_to','movements_norm']]

    #остановки-суперсайты
    stops_supers = pd.read_csv(path('stops_supers.csv'), sep = ';', encoding='cp1251')

    matrix = pd.merge(matrix, stops_supers, how='inner', left_on = ['stop_id_from'], right_on = ['stop_id'])
    matrix = pd.merge(matrix, stops_supers, how='inner', left_on = ['stop_id_to'], right_on = ['stop_id'])
    matrix = matrix[['super_site_x','super_site_y','movements_norm']].rename(columns = {'super_site_x':'super_site_from',
                                                                                       'super_site_y':'super_site_to'})
    matrix = matrix.groupby(['super_site_from','super_site_to']).sum().reset_index()

    supers_Moscow = pd.read_csv(path('supers_Mercator.csv'), sep = ';')
    supers_Moscow = supers_Moscow.drop_duplicates()

    links = pd.merge(matrix, supers_Moscow, how = 'inner', 
                  left_on = ['super_site_from'], right_on=['super_site']).rename(columns={'X':'X_from','Y':'Y_from'})
    links = pd.merge(links,  supers_Moscow, how = 'inner', 
               left_on = ['super_site_to'], right_on=['super_site']).rename(columns={'X':'X_to','Y':'Y_to'})
    links = links[['super_site_from','super_site_to','movements_norm','X_from','Y_from','X_to','Y_to']]
    links['movements_norm'] = links['movements_norm']/3
    links['movements_norm'] = round(links['movements_norm'],0)

    #сайты Тушино из
    supers_T = pd.read_csv(path('supersites_Tushino.csv'), sep = ';')

    links = pd.merge(links, supers_T, how='inner',left_on=['super_site_from'], right_on=['super_site'])
    links = links[links['movements_norm']>10]









    source_from = ColumnDataSource(data = dict(
                            X_from=list(links['X_from'].values), 
                            Y_from=list(links['Y_from'].values),
                            size=list(links['movements_norm'].values),
                            X_to=list(links['X_to'].values), 
                            Y_to=list(links['Y_to'].values)
    ))


    source_to = ColumnDataSource(data = dict(
                            X_from=list(links['X_from'].values), 
                            Y_from=list(links['Y_from'].values),
                            size=list(links['movements_norm'].values),
                            X_to=list(links['X_to'].values), 
                            Y_to=list(links['Y_to'].values)
    ))



    source_from2 = ColumnDataSource(data = dict(
                            X_from=list(links['X_from'].values), 
                            Y_from=list(links['Y_from'].values),
                            size=list(links['movements_norm'].values),
                            X_to=list(links['X_to'].values), 
                            Y_to=list(links['Y_to'].values)
    ))


    source_to2 = ColumnDataSource(data = dict(
                            X_from=list(links['X_from'].values), 
                            Y_from=list(links['Y_from'].values),
                            size=list(links['movements_norm'].values),
                            X_to=list(links['X_to'].values), 
                            Y_to=list(links['Y_to'].values)
    ))


    source_from_labels = ColumnDataSource(data = dict(
                            X_from=list(links[['X_from', 'Y_from','super_site_from']].drop_duplicates()['X_from'].values), 
                            Y_from=list(links[['X_from', 'Y_from','super_site_from']].drop_duplicates()['Y_from'].values),
                            label=list(links[['X_from', 'Y_from','super_site_from']].drop_duplicates()['super_site_from'].values)
    ))






    hover = HoverTool(tooltips=[('site_id','@label')])

    toolList = ['lasso_select', 'tap', 'reset', 'save', 'pan','wheel_zoom']

    p = figure(x_range=(4157975.01546188769862056 , 4173827.06850233720615506), y_range=(7521739.63348639197647572,  7533621.55124872922897339),
              x_axis_type="mercator", y_axis_type="mercator", tools=toolList)
    p.add_tile(CARTODBPOSITRON)

    p.add_layout(Title(text='Фильтр корреспонденций "ИЗ"', text_font_size='10pt'), 'above')

    p.circle(x = 'X_from',
             y = 'Y_from',
             source=source_from_labels,
            fill_color='black',
            size=5)

    r = p.circle(x = 'X_from',
             y = 'Y_from',
             source=source_from,
            fill_color='navy',
            size=10,
            fill_alpha = 1,
            nonselection_fill_alpha=0.4)



    p_to = figure(x_range=(4157975.01546188769862056 , 4173827.06850233720615506), y_range=(7521739.63348639197647572,  7533621.55124872922897339),
              x_axis_type="mercator", y_axis_type="mercator", tools=toolList)
    p_to.add_tile(CARTODBPOSITRON)
    t = p_to.circle(x = 'X_to', y = 'Y_to', fill_color='red', fill_alpha = 0.6, 
                                line_color='red', line_alpha = 0.8, size=6 , source = source_to)

    ds = r.data_source





    def callback(attrname, old, new):
        idx = source_from.selected.indices
        print("Indices of selected circles: ", idx)
        print("Lenght of selected circles: ", len(idx))



        #таблица с выбранными индексами 
        df = pd.DataFrame(data=ds.data).iloc[idx]
        #сумма movements по выделенным индексам
        aa = df.groupby(['X_to','Y_to'])['size'].transform(sum)


        p_to = figure(x_range=(4157975.01546188769862056 , 4173827.06850233720615506), 
                      y_range=(7521739.63348639197647572,  7533621.55124872922897339),
                      x_axis_type="mercator", y_axis_type="mercator", tools=toolList)
        p_to.add_tile(CARTODBPOSITRON)
        t = p_to.circle(x = 'X_to', y = 'Y_to', fill_color='red', fill_alpha = 0.6, 
                                line_color='red', line_alpha = 0.8, size=6 , source = source_to)


        if not idx: #если пустое выделение

            layout1.children[1] = p_to #обновить график справа

        else: #если не пустое выделение


            for x in idx: #для каждого выделенного индекса рисуем site_to и его параметры

                new_data = dict()
                new_data['x'] = [ds.data['X_to'][x]]
                new_data['y'] = [ds.data['Y_to'][x]]
                new_data['size'] = [aa[x]]


                t = p_to.circle(x = [], y = [], fill_color='orange', fill_alpha = 0.6, 
                                line_color='red', line_alpha = 0.8, size=[] )
                tds=t.data_source
                tds.data = new_data

                #текст

                new_data_text = dict()
                new_data_text['x'] = [ds.data['X_to'][x]]
                new_data_text['y'] = [ds.data['Y_to'][x]]
                new_data_text['text'] = [aa[x]]


                l = p_to.text(x = [], y = [], text_color='black', text =[], text_font_size='8pt',
                             text_font_style = 'bold')
                lds=l.data_source
                lds.data = new_data_text


                layout1.children[1] = p_to #обновить график справа





    source_from.selected.on_change('indices', callback)



    p2 = figure(x_range=(4157975.01546188769862056 , 4173827.06850233720615506), y_range=(7521739.63348639197647572,  7533621.55124872922897339),
              x_axis_type="mercator", y_axis_type="mercator", tools=toolList)
    p2.add_tile(CARTODBPOSITRON)


    p2.add_layout(Title(text='Фильтр корреспонденций "В"', text_font_size='10pt'), 'above')


    r2 = p2.circle(x = 'X_to',
             y = 'Y_to',
             source=source_to2,
            fill_color='navy',
            size=10,
            fill_alpha = 1,
            nonselection_fill_alpha=0.4)



    p_from = figure(x_range=(4157975.01546188769862056 , 4173827.06850233720615506), y_range=(7521739.63348639197647572,  7533621.55124872922897339),
              x_axis_type="mercator", y_axis_type="mercator", tools=toolList)
    p_from.add_tile(CARTODBPOSITRON)
    t2 = p_from.circle(x = 'X_from', y = 'Y_from', fill_color='red', fill_alpha = 0.6, 
                                line_color='red', line_alpha = 0.8, size=6 , source = source_from2)




    ds2 = r2.data_source

    def callback2(attrname, old, new):
        idx = source_to2.selected.indices
        print("Indices of selected circles: ", idx)
        print("Lenght of selected circles: ", len(idx))



        #таблица с выбранными индексами 
        df = pd.DataFrame(data=ds2.data).iloc[idx]
        #сумма movements по выделенным индексам
        aa = df.groupby(['X_from','Y_from'])['size'].transform(sum)


        p_from = figure(x_range=(4157975.01546188769862056 , 4173827.06850233720615506), 
                      y_range=(7521739.63348639197647572,  7533621.55124872922897339),
                      x_axis_type="mercator", y_axis_type="mercator", tools=toolList)
        p_from.add_tile(CARTODBPOSITRON)
        t2 = p_from.circle(x = 'X_from', y = 'Y_from', fill_color='red', fill_alpha = 0.6, 
                                line_color='red', line_alpha = 0.8, size=6 , source = source_from2)


        if not idx: #если пустое выделение

            layout2.children[1] = p_from #обновить график справа

        else: #если не пустое выделение


            for x in idx: #для каждого выделенного индекса рисуем site_to и его параметры

                new_data = dict()
                new_data['x'] = [ds2.data['X_from'][x]]
                new_data['y'] = [ds2.data['Y_from'][x]]
                new_data['size'] = [aa[x]]


                t2 = p_from.circle(x = [], y = [], fill_color='orange', fill_alpha = 0.6, 
                                line_color='red', line_alpha = 0.8, size=[] )
                tds2=t2.data_source
                tds2.data = new_data

                #текст

                new_data_text = dict()
                new_data_text['x'] = [ds2.data['X_from'][x]]
                new_data_text['y'] = [ds2.data['Y_from'][x]]
                new_data_text['text'] = [aa[x]]


                l2 = p_from.text(x = [], y = [], text_color='black', text =[], text_font_size='8pt',
                             text_font_style = 'bold')
                lds2=l2.data_source
                lds2.data = new_data_text


                layout2.children[1] = p_from #обновить график справа





    source_to2.selected.on_change('indices', callback2)


    layout1 = layout.row(p,p_to)
    layout2 = layout.row(p2,p_from)

    # Create Tabs
    line = layout.column(layout1,layout2)


    doc.add_root(line)


# In[ ]:





# In[ ]:





# In[ ]:





# In[ ]:





# In[4]:


# apps = {'/': Application(FunctionHandler(make_document))}

# server = Server(apps, port=5001)
# server.start()


# In[5]:


# server = server({'/': make_document}, num_procs=1)
# server.start()

# if __name__ == '__main__':
#     print('Opening Bokeh application on http://localhost:5006/')

# server.io_loop.add_callback(server.show, "/")
#server.io_loop.start()


# In[6]:


apps = {'/': Application(FunctionHandler(make_document))}

server = server(apps, port=5001)
server.start()


if __name__ == '__main__':
    print('Opening Bokeh application on http://localhost:5006/')

server.io_loop.add_callback(server.show, "/")


# In[ ]:





# In[7]:


# handler = FunctionHandler(make_document)
# app = Application(handler)


# In[8]:


# doc = app.create_document()
# show(app, notebook_url="localhost:8888")


# In[ ]:

Opening Bokeh application on http://localhost:5006/
Indices of selected circles:  [204, 205, 206, 207, 208]
Lenght of selected circles:  5
Indices of selected circles:  []
Lenght of selected circles:  0
Indices of selected circles:  [176, 177, 178, 179, 180]
Lenght of selected circles:  5
Indices of selected circles:  []
Lenght of selected circles:  0
Indices of selected circles:  [76, 77, 78, 79, 80]
Lenght of selected circles:  5
Indices of selected circles:  []
Lenght of selected circles:  0
Indices of selected circles:  [246]
Lenght of selected circles:  1
Indices of selected circles:  [237, 238, 246]
Lenght of selected circles:  3
Indices of selected circles:  [220, 221, 222, 223, 224, 237, 238, 246]
Lenght of selected circles:  8
Indices of selected circles:  []
Lenght of selected circles:  0
Indices of selected circles:  [85, 86]
Lenght of selected circles:  2
Indices of selected circles:  []
Lenght of selected circles:  0
Indices of selected circles:  [195]
Lenght of selected c

In [2]:
server {
    listen 80 default_server;
    server_name _;

    access_log  /tmp/bokeh.access.log;
    error_log   /tmp/bokeh.error.log debug;

    location / {
        proxy_pass http://127.0.0.1:5100;
        proxy_set_header Upgrade $http_upgrade;
        proxy_set_header Connection "upgrade";
        proxy_http_version 1.1;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header Host $host:$server_port;
        proxy_buffering off;
    }

}

SyntaxError: invalid syntax (<ipython-input-2-21c179f0970f>, line 1)